# General Usage

In [1]:
from pprint import pprint
import tdameritrade as td
import os
import json

## Authentication



In [2]:
# get refreshtoken from file
refreshtoken = open(os.path.expanduser('~/.r_token'), 'r')
line = refreshtoken.readline()
refreshtoken.close()

client_id = os.getenv('TDAMERITRADE_CLIENT_ID')
account_id = os.getenv('TDAMERITRADE_ACCOUNT_ID')

# authenticate and initialize TD Client
td_client = td.TDClient(client_id=client_id, refresh_token=line)



## API Examples

In [3]:
import pandas as pd
pd.DataFrame(td_client.history('AAPL')['candles']).head()

,open,high,low,close,volume,datetime
0,136.41,136.50,136.38,136.50,15760,1612958400000
1,136.47,136.50,136.43,136.43,2762,1612958460000
2,136.43,136.48,136.43,136.48,1416,1612958520000
3,136.47,136.50,136.45,136.50,5083,1612958580000
4,136.47,136.47,136.44,136.44,1776,1612958640000


### Getting Option Data

In [4]:
#options = td_client.options('BAX')
#expiry_dates = list(options['callExpDateMap'].keys())
#expiry = expiry_dates[0]
#expiry = '2021-02-19:9'
#print(f"Expiry Date: {expiry}")
#pprint(expiry_dates)
import pandas as pd
#strike_prices = list(options['callExpDateMap'][expiry].keys())
#strike = strike_prices[0]
#strike = '70.0'
#print(f"Strike Price: {strike}")
#pprint(strike_prices)

#pprint(options['callExpDateMap'][expiry][strike])
#options = td_client.optionsDF('BAX')
options = td_client.optionsDF('CTXS')

df = options[['symbol','delta','expirationDate']]
#expiryDate = 
#options[(options['delta'] > 0.95) & ('expirationDate' == expiryDate)]
df[(df['delta'] > 0.95)]

options2 = td_client.optionsDF('BAX')
df2 = options2[['symbol','delta','expirationDate']]
df2
#df2['delta'] > 0.95)
#df[['symbol','delta','expirationDate']]
#pd.DataFrame(options['symbol','delta','expirationDate'])


,symbol,delta,expirationDate
0,BAX_022621C40,0.94,2021-02-26 21:00:00
1,BAX_022621C45,0.927,2021-02-26 21:00:00
2,BAX_022621C50,0.912,2021-02-26 21:00:00
3,BAX_022621C55,0.894,2021-02-26 21:00:00
4,BAX_022621C60,0.869,2021-02-26 21:00:00
...,...,...,...
773,BAX_012023P105,-0.773,2023-01-20 21:00:00
774,BAX_012023P110,-0.82,2023-01-20 21:00:00
775,BAX_012023P115,-0.845,2023-01-20 21:00:00
776,BAX_012023P120,-0.883,2023-01-20 21:00:00


## Getting Account Positions

In [11]:
# PRIVATE account info
# pprint(td_client.accounts(positions=True))

## Placing an Order

### Creating order body for Place Order API

In [14]:
from tdameritrade.orders.order_builder import build_buy_market_stock_order

# Prints function docstring
# build_buy_market_stock_order?

In [15]:
symbol = "HUSA"  # Cheap stock for example
quantity = 1
order = build_buy_market_stock_order(symbol, quantity)

In [16]:
pprint(order.asdict())

{'duration': 'DAY',
 'orderLegCollection': [{'instruction': 'BUY',
                         'instrument': {'assetType': 'EQUITY',
                                        'symbol': 'HUSA'},
                         'quantity': 1}],
 'orderStrategyType': 'SINGLE',
 'orderType': 'MARKET',
 'session': 'NORMAL'}


### Submitting Order

In [18]:
# Without this, it fails with a 415.
# This may screw up other API calls, so it may have to be reset if you want to try other ones.
td_client.session.headers = {
    "Content-Type": "application/json"
}

In [19]:
# This will fail to return because the place order API does not return a JSONserializable response on success
# BUT the order will still go through.  Checked on the site.  You can also check your active orders below.

try:
    response = td_client.placeOrder(account_id, order.json())
    pprint(response)
except json.decoder.JSONDecodeError as err:
    print(err)


ValidationError: -1: An unknown error occurred

### Getting out Orders

In [12]:
# Load account info to get account ID for get orders
#td_client.accounts()

#account_id = td_client.accountIds[0]

In [13]:
# Print out orders
my_orders = td_client.orders(accountId=account_id)
pprint(my_orders)

[]


### Deleting Orders

In [19]:
# This deletes all WORKING orders

# for order in my_orders:
#     if order["status"] == Status.WORKING and order["cancelable"]:
#         response = td_client.cancelOrder(account_id, order["orderId"])
#         pprint(response)